In [ ]:
import osmium as osm
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.cluster import DBSCAN
import numpy as np
import os
from collections import Counter

# Setze den Pfad zum Dataset
dataset_path = os.path.join("Datasets", "wenigumstadt.osm")
filename = os.path.basename(dataset_path)
title = filename.split('.')[0].capitalize()

In [ ]:
# Datensatz um alle Daten auszulesen
class OSMHandler(osm.SimpleHandler):
    def __init__(self):
        osm.SimpleHandler.__init__(self)
        self.osm_data = []

    def tag_inventory(self, elem, elem_type, lon=None, lat=None, node_ids=None):
        if len(elem.tags) == 0:
            # Für Knoten ohne Tags
            self.osm_data.append([elem_type, 
                                  elem.id, 
                                  elem.visible, 
                                  0, 
                                  None, 
                                  None, 
                                  lon, 
                                  lat, 
                                  node_ids])
        else:
            for tag in elem.tags:
                self.osm_data.append([elem_type, 
                                      elem.id,
                                      elem.visible,
                                      len(elem.tags),
                                      tag.k, 
                                      tag.v, lon, lat, node_ids])

    def node(self, n):
        self.tag_inventory(n, "node", lon=n.location.lon, lat=n.location.lat)

    def way(self, w):
        node_ids = [n.ref for n in w.nodes]  
        self.tag_inventory(w, "way", node_ids=node_ids)

# Daten auslesen
osmhandler = OSMHandler()
osmhandler.apply_file(dataset_path)

# DataFrame erstellen
data_colnames = ['type', 'id', 'visible', 'ntags', 'tagkey', 'tagvalue', 'longitude', 'latitude', 'node_ids']
df_osm = pd.DataFrame(osmhandler.osm_data, columns=data_colnames)
df_osm

In [ ]:
# Datensatz für Straßen (Kompletter Datensatz)
ways_filter = df_osm[(df_osm['type'] == 'way') &
                        (df_osm['tagkey'] == 'highway')]

ways_filter

In [ ]:
# Datensatz auf nur Kreuzungspunkte reduzieren (Komplett)
all_node_ids = []
for node_ids_list in ways_filter['node_ids']:
    if node_ids_list is not None:
        all_node_ids.extend(node_ids_list)

node_reference_count = Counter(all_node_ids)

nodes_referenced_multiple = {node_id for node_id, count in node_reference_count.items() if count > 1}

df_nodes_filtered_multiple = df_osm[(df_osm['type'] == 'node') & (df_osm['id'].isin(nodes_referenced_multiple))]

filtered_data_multiple = pd.concat([ways_filter, df_nodes_filtered_multiple], ignore_index=True)

filtered_data_multiple

In [ ]:
# Knoten zusammenfügen, die nah beieinander sind

# Knoten in Cluster zusammenfassen
coords = df_nodes_filtered_multiple[['longitude', 'latitude']].values
eps_distance = 0.0004
clustering = DBSCAN(eps=eps_distance, min_samples=1).fit(coords)

# Cluster-Labels zuweisen
df_nodes_filtered_multiple['cluster'] = clustering.labels_

# Cluster-Vertreter bestimmen
cluster_representatives = df_nodes_filtered_multiple.groupby('cluster')['id'].min().to_dict()

# Ersetzungs-Dictionary für alle Knoten -> Cluster-Vertreter
node_to_representative = {}
for node_id, cluster in df_nodes_filtered_multiple[['id', 'cluster']].values:
    if cluster in cluster_representatives:
        node_to_representative[node_id] = cluster_representatives[cluster]

print(f"node_to_representative für ID 6251639256: {node_to_representative.get(6251639256)}")

def replaces_node_ids(node_ids):
    if node_ids is None:
        return None
    # Knoten-Id in node_ids durch zugehörige Cluster-Vetreter-Id ersetzen
    replaced_node_ids = [node_to_representative.get(node_id, node_id) for node_id in node_ids]
    print(f"Original: {node_ids} -> Ersetzt: {replaced_node_ids}") 
    return replaced_node_ids

ways_filter['node_ids'] = ways_filter['node_ids'].apply(replaces_node_ids)

ways_filter['node_ids'] = ways_filter['node_ids'].apply(lambda x: list(dict.fromkeys(x)))

df_cluster_nodes = df_nodes_filtered_multiple[df_nodes_filtered_multiple['id'].isin(cluster_representatives.values())]
df_cluster_nodes = df_cluster_nodes.drop(columns='cluster')

filtered_data_combined = pd.concat([ways_filter, df_cluster_nodes], ignore_index=True)
filtered_data_combined

In [ ]:
test = filtered_data_combined[filtered_data_combined["id"] == 3190521520]
test

In [ ]:
# Länge der einzelnen Straßen ausrechnen
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    # Dezimalzahlen in Radianten umwandeln
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine Formel
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius der Erde in Kilometer
    return c * r * 1000

# Funktion zur Berechnung der Straßenlänge basierend auf den referenzierten Knoten
def calculate_road_length(way_row, df_nodes):
    first_node_id = way_row['node_ids'][0]
    last_node_id = way_row['node_ids'][-1]

    # Überprüfe, ob der erste Knoten im DataFrame existiert
    first_node = df_nodes[df_nodes['id'] == first_node_id]
    if first_node.empty:
        print(f"Warnung: Erster Knoten mit ID {first_node_id} nicht gefunden!")
        return 0 

    # Überprüfe, ob der letzte Knoten im DataFrame existiert
    last_node = df_nodes[df_nodes['id'] == last_node_id]
    if last_node.empty:
        print(f"Warnung: Letzter Knoten mit ID {last_node_id} nicht gefunden!")
        return 0 

    # Berechne die Entfernung zwischen den Knoten
    length = haversine(first_node['longitude'].values[0], first_node['latitude'].values[0],
                       last_node['longitude'].values[0], last_node['latitude'].values[0])
    return length

filtered_data_combined['road_length'] = filtered_data_combined[filtered_data_combined['type'] == 'way'].apply(calculate_road_length, axis=1, df_nodes=df_cluster_nodes)
filtered_data_combined

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

G = nx.Graph()

# Knoten zum Graphen hinzufügen
for index, row in filtered_data_combined.iterrows():  
    if pd.notnull(row['longitude']) and pd.notnull(row['latitude']):
        G.add_node(row['id'], pos=(row['longitude'], row['latitude']))

# Kanten zum Graphen hinzufügen
for index, way in ways_filter.iterrows():
    node_ids_list = way['node_ids']
    if node_ids_list is not None:
        filtered_node_ids_list = [node_id for node_id in node_ids_list if G.has_node(node_id)]
        if len(filtered_node_ids_list) > 1:
            for i in range(len(filtered_node_ids_list) - 1):
                is_dashed_highway = way['tagkey'] == 'highway' and way['tagvalue'] not in ['residential', 'unclassified', 'service', 'tertiary', 'secondary']
                G.add_edge(
                    filtered_node_ids_list[i], filtered_node_ids_list[i + 1],
                    street_name=way['tagvalue'],
                    is_dashed_highway=is_dashed_highway
                )

pos = nx.get_node_attributes(G, 'pos')

plt.figure(figsize=(12, 8))

# Normale Kanten zeichnen
edges_normal = [(u, v) for u, v, d in G.edges(data=True) if not d.get('is_dashed_highway')]
nx.draw_networkx_edges(G, pos, edgelist=edges_normal, edge_color='gray', width=1.5)

# Gestrichelte Kanten zeichnen
edges_dashed = [(u, v) for u, v, d in G.edges(data=True) if d.get('is_dashed_highway')]
nx.draw_networkx_edges(G, pos, edgelist=edges_dashed, edge_color='gray', width=1.5, style='--')

nx.draw_networkx_nodes(G, pos, node_size=50, node_color='blue')

# Optional: Kantenbeschriftungen mit Straßennamen anzeigen
# edge_labels = nx.get_edge_attributes(G, 'street_name')
# nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8, font_color='red')

plt.title(title)
plt.xlabel("Längengrad")
plt.ylabel("Breitengrad")
plt.show()

In [ ]:
edge_count = G.number_of_edges()
print("Anzahl der Kanten:", edge_count)

In [ ]:
import plotly.graph_objects as go

node_degrees = dict(G.degree()) 

node_x = []
node_y = []
node_text = []  
node_color = [] 

for index, row in filtered_data_combined.iterrows():
    if pd.notnull(row['longitude']) and pd.notnull(row['latitude']):
        node_x.append(row['longitude'])
        node_y.append(row['latitude'])
        
        hover_info = f"ID: {row['id']}<br>Tag Key: {row['tagkey']}<br>Tag Value: {row['tagvalue']}<br>Degree: {node_degrees[row['id']]}"
        node_text.append(hover_info)
        
        node_color.append(node_degrees[row['id']])

edge_x = []
edge_y = []
dashed_edge_x = []
dashed_edge_y = []
edge_street_names = []

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]

    # Überprüfen, ob der Rand highway != 'residual' ist
    is_dashed_highway = False
    street_name = ""
    for street in ways_filter.itertuples():
        if edge[0] in street.node_ids and edge[1] in street.node_ids:
            street_name = street.tagvalue
            # Highway-Kanten, die nicht 'residual' sind, werden gestrichelt
            if street.tagkey == "highway" and street.tagvalue not in ['residential', 'unclassified', 'service', 'tertiary', 'secondary']:
                is_dashed_highway = True
            break

    if is_dashed_highway:
        dashed_edge_x.extend([x0, x1, None])
        dashed_edge_y.extend([y0, y1, None])
    else:
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    # Berechnung der Mittelpunkte für die Straßennamen
    midpoint_x = (x0 + x1) / 2
    midpoint_y = (y0 + y1) / 2
    edge_street_names.append((midpoint_x, midpoint_y, street_name))


# Plot für durchgezogene Kanten
solid_edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines'
)

# Plot für gestrichelte Kanten (highways)
dashed_edge_trace = go.Scatter(
    x=dashed_edge_x, y=dashed_edge_y,
    line=dict(width=0.5, color='#888', dash='dash'),
    hoverinfo='none',
    mode='lines'
)

# Plot für Knoten
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        color=node_color,
        size=10,
        colorbar=dict(
            thickness=15,
            title='Knoten Verbindungen',
            xanchor='left',
            titleside='right'
        )),
    text=node_text
)

# Hinzufügen der Straßennamen zwischen den Knoten
street_name_trace = go.Scatter(
    x=[name[0] for name in edge_street_names],
    y=[name[1] for name in edge_street_names],
    text=[name[2] for name in edge_street_names],
    mode='text',
    textfont=dict(
        size=12,
        color='darkblue'
    ),
    hoverinfo='text'
)

layout = go.Layout(
    title='Großostheim',
    titlefont=dict(size=16),
    showlegend=False,
    hovermode='closest',
    margin=dict(b=0, l=0, r=0, t=0),
    annotations=[dict(
        text=title,
        showarrow=False,
        xref="paper", yref="paper",
        x=0.005, y=-0.002)],
    xaxis=dict(showgrid=False, zeroline=False, visible=False),
    yaxis=dict(showgrid=False, zeroline=False, visible=False)
)

fig = go.Figure(data=[solid_edge_trace, dashed_edge_trace, node_trace], layout=layout)

fig.show()


In [ ]:
from matplotlib.ticker import MaxNLocator

degrees = list(node_degrees.values())
degree_sequence = sorted(degrees, reverse=True)

unique_degrees, counts = np.unique(degrees, return_counts=True)

fig = plt.figure(figsize=(8, 6))

plt.bar(unique_degrees, counts, color='skyblue', edgecolor='black')
plt.title("Degree Histogram of" + " " + title)
plt.xlabel("Degree")
plt.ylabel("# of Nodes")

plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))

plt.tight_layout()

plt.show()

In [ ]:
# Filtere die Straßen mit gültigen Längen (nicht 0 oder NaN)
valid_road_lengths = filtered_data_combined['road_length'].dropna()
valid_road_lengths = valid_road_lengths[valid_road_lengths > 0]

# Erstelle das Histogramm
plt.figure(figsize=(10, 6))
plt.hist(valid_road_lengths, bins=30, color='skyblue', edgecolor='black')
plt.title('Verteilung der Straßenlänge'  + " " + title)
plt.xlabel('Straßenlänge (Meter)')
plt.ylabel('Anzahl der Straßen')
plt.grid(True)
plt.show()

In [ ]:
# Plot with Degree Diagrams
image_folder = 'Degree of various places'
images = [file for file in os.listdir(image_folder) if file.endswith(('png'))]

# Number of images
num_images = len(images)

cols = 3
rows = num_images // cols + (num_images % cols > 0)

fig, axs = plt.subplots(rows, cols, figsize=(15, 5 * rows))

for i, img_name in enumerate(images):
    img_path = os.path.join(image_folder, img_name)
    img = mpimg.imread(img_path)

    ax = axs[i // cols, i % cols] if rows > 1 else axs[i % cols]
    ax.imshow(img)
    ax.axis('off')
    ax.set_title(img_name)

for j in range(i + 1, rows * cols):
    fig.delaxes(axs.flatten()[j])

plt.tight_layout()
plt.show()

In [ ]:
# Plot with length Diagrams
image_folder = 'Street length of various places'
images = [file for file in os.listdir(image_folder) if file.endswith(('png'))]

# Number of images
num_images = len(images)

cols = 3
rows = num_images // cols + (num_images % cols > 0)

fig, axs = plt.subplots(rows, cols, figsize=(15, 5 * rows))

for i, img_name in enumerate(images):
    img_path = os.path.join(image_folder, img_name)
    img = mpimg.imread(img_path)

    ax = axs[i // cols, i % cols] if rows > 1 else axs[i % cols]
    ax.imshow(img)
    ax.axis('off')
    ax.set_title(img_name)

for j in range(i + 1, rows * cols):
    fig.delaxes(axs.flatten()[j])

plt.tight_layout()
plt.show()